In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/phi2-mentalhealth.zip

Archive:  /content/drive/MyDrive/phi2-mentalhealth.zip
replace phi2-mentalhealth/special_tokens_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!ls /content/phi2-mentalhealth

adapter_config.json	   checkpoint-400  README.md
adapter_model.safetensors  checkpoint-500  special_tokens_map.json
added_tokens.json	   checkpoint-600  tokenizer_config.json
checkpoint-100		   checkpoint-700  tokenizer.json
checkpoint-200		   checkpoint-750  vocab.json
checkpoint-300		   merges.txt


In [4]:
# STEP 1: Install required packages
!pip install flask flask-cors
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz

# STEP 2: Add your authtoken (Get from https://dashboard.ngrok.com/get-started/your-authtoken)
!./ngrok config add-authtoken 300Sm5f3rBJ3vGzFad5QkjM7uYZ_5pu5JbHeB9yhzi7XyHxhS

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 🧠 Load fine-tuned Phi-2 model
model_path = "/content/phi2-mentalhealth"  # update if needed

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
model.eval()

# 🧠 Flask API setup
app = Flask(__name__)
CORS(app)

# Store a basic in-memory chat history (can be made user-specific later)
chat_history = []

@app.route("/chat", methods=["POST"])
def chat():
    global chat_history

    user_message = request.json.get("message", "")

    # Format the chat history into a prompt
    prompt = ""
    for user_msg, bot_msg in chat_history:
        prompt += f"<|user|>\n{user_msg}\n<|assistant|>\n{bot_msg}\n"
    prompt += f"<|user|>\n{user_message}\n<|assistant|>\n"

    # Tokenize input for model
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded_output.split("<|assistant|>")[-1].strip()

    # Append to chat history
    chat_history.append((user_message, response))

    return jsonify({"response": response})

# Run Flask in background
def run_flask():
    app.run(port=5000)

threading.Thread(target=run_flask).start()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


In [6]:
# STEP 4: Start ngrok and show public URL
import subprocess
import time
import requests

# Start ngrok tunnel
ngrok = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(5)  # wait for tunnel to initialize

# Get public URL
try:
    response = requests.get("http://localhost:4040/api/tunnels")
    tunnels = response.json()['tunnels']
    public_url = tunnels[0]['public_url']
    print("🚀 Your React app should send POST requests to:", public_url + "/chat")
except Exception as e:
    print("❌ Could not fetch ngrok URL:", e)

🚀 Your React app should send POST requests to: https://2fde193ae065.ngrok-free.app/chat


In [7]:
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # <-- This enables CORS for all routes